## Imports and Reading Datasets

In [1]:
from uszipcode import SearchEngine
import pandas as pd
import numpy as np
import random

search = SearchEngine(simple_zipcode=True)
df = pd.read_csv('./datasets/iowaliquor.csv')[['Date', 'Store Name', 'Zip Code', 'Sale (Dollars)', 'County']]

/home/ubuntu/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (6,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
# get 2016 election data and filter for Iowa results only 
election_df = pd.read_csv('./datasets/usa-2016-presidential-election-by-county.csv', sep=';')
election_df = election_df[election_df['State'] == 'Iowa']

In [1]:
# delete this
election_df.head()['Less Than High School Diploma']

NameError: name 'election_df' is not defined

## Fill Missing Zip Codes

In [4]:
unqs = df[df["Zip Code"].isna()]["Store Name"].unique() # get unique store names
cities = [store.split("/")[-1].strip().lower() for store in unqs if "/" in store] # extract city name
if "mlk" in cities:
    cities.remove("mlk")
subset = df[df["Store Name"].str.contains("/") & (df["Zip Code"].isna())] 
city_dict = {}
for city in set(cities): # get zip code from rows with cities in store name
    city_dict[city] = search.by_city_and_state(city, "iowa")[0].zipcode
city_dict["mlk"] = '50310'
df.loc[subset.index, 'Zip Code'] = subset["Store Name"].str.split("/").str[-1].str.strip().str.lower().map(city_dict) 

## Aggregate and Get Zip Code Statistics

In [5]:
df["YearMonth"] = df["Date"].str.split("/").str[2] + "-" + df["Date"].str.split("/").str[0]

df = df[df["Zip Code"].notna()]
df["Zip Code"] = df["Zip Code"].replace("712-2", 51529) 
df["Zip Code"] = df["Zip Code"].astype(int) 

countyDict = {}
popDict = {}

for zc in df["Zip Code"].unique():
    stats = search.by_zipcode(zc)
    if stats.state == "IA":
        countyDict[zc] = stats.county
        popDict[zc] = stats.population

In [6]:
df.head()

,Date,Store Name,Zip Code,Sale (Dollars),County,YearMonth
0,10/04/2012,Kum & Go #573 / SE 14th DM,50315,17.25,Polk,2012-10
1,12/20/2014,Roy's Foodland,52332,27.66,Benton,2014-12
2,09/09/2013,Wal-Mart 1528 / Cedar Rapids,52402,134.76,Linn,2013-09
3,12/12/2012,No Frills Supermarkets #803 / Glenwo,51534,495.60,Mills,2012-12
4,10/14/2015,Hy-Vee Food Store / Sioux City,51106,121.56,Woodbury,2015-10


In [7]:
totalZip = pd.DataFrame(df.groupby(["YearMonth", "Zip Code"])["Sale (Dollars)"].sum())
totalZip.reset_index(inplace=True)
totalZip["Population"] = totalZip["Zip Code"].map(popDict)

## County Aggregate & Add Election Data

In [8]:
totalZip["County"] = totalZip["Zip Code"].map(countyDict)
totalZip.dropna(inplace=True)
totalCounty = pd.DataFrame(totalZip.groupby(["YearMonth", "County"])["Sale (Dollars)"].sum())
totalCounty.reset_index(inplace=True)
totalCounty["Month"] = totalCounty["YearMonth"].str.split("-").str[-1]

election_df["County"] = election_df["County"].str.replace(', Iowa', "")
totalCounty = totalCounty.merge(election_df, on="County")
totalCounty["SalesPerCapita"] = totalCounty["Sale (Dollars)"] / totalCounty["Total Population"]

In [9]:
totalZip.head()

,YearMonth,Zip Code,Sale (Dollars),Population,County
0,2012-01,50002,2865.89,1297.0,Adair County
1,2012-01,50003,12127.81,6594.0,Dallas County
2,2012-01,50006,8519.98,1676.0,Hardin County
3,2012-01,50009,90278.27,16136.0,Polk County
4,2012-01,50010,386395.66,29359.0,Story County


In [10]:
totalCounty.head()

,YearMonth,County,Sale (Dollars),Month,State,ST,Fips,Precincts,Votes,Democrats 08 (Votes),...,total12,other08,other12,Other12 Frac,Other08 Frac,Rep12 Frac2,Rep08 Frac2,Dem12 Frac2,Dem08 Frac2,SalesPerCapita
0,2012-01,Adair County,15988.30,01,Iowa,IA,19001,6.0,3759.0,1924.0,...,3996.0,69.0,92.0,0.023023,0.017024,0.541496,0.517068,0.458504,0.482932,2.098753
1,2012-02,Adair County,17596.63,02,Iowa,IA,19001,6.0,3759.0,1924.0,...,3996.0,69.0,92.0,0.023023,0.017024,0.541496,0.517068,0.458504,0.482932,2.309875
2,2012-03,Adair County,14380.75,03,Iowa,IA,19001,6.0,3759.0,1924.0,...,3996.0,69.0,92.0,0.023023,0.017024,0.541496,0.517068,0.458504,0.482932,1.887733
3,2012-04,Adair County,18161.12,04,Iowa,IA,19001,6.0,3759.0,1924.0,...,3996.0,69.0,92.0,0.023023,0.017024,0.541496,0.517068,0.458504,0.482932,2.383975
4,2012-05,Adair County,18881.41,05,Iowa,IA,19001,6.0,3759.0,1924.0,...,3996.0,69.0,92.0,0.023023,0.017024,0.541496,0.517068,0.458504,0.482932,2.478526


## Unemployment Data

In [11]:
totalCounty = pd.read_csv("totalCounty.csv").drop("Unnamed: 0", axis=1)
totalCounty["Year"] = totalCounty["YearMonth"].str.split("-").str[0]
unemployment_df = pd.read_csv("./datasets/iowa_unemployment.csv")
unemployment_df.rename(columns={"County Name": "County"}, inplace=True)
unemployment_df["Year"] = unemployment_df["Year"].astype(str)
unemployment_df["County"] = unemployment_df["County"] + " County"
totalCounty = pd.merge(totalCounty, unemployment_df,  how='left', left_on=['County','Year'], right_on = ['County','Year'])

In [12]:
unemployment_df.head()

,Year,County,Benefits Paid,Weeks Compensated,Recipients,First Payments,Final Payments,County FIP,GNIS Feature ID,Primary Lat Dec,Primary Long Dec,Primary County Coordinates
0,2019,Worth County,971612,2529,290,234,32,19195,465286,43.377398,-93.260848,POINT (-93.2608476 43.3773984)
1,2019,Woodbury County,10148200,25818,2505,2060,463,19193,465285,42.389717,-96.044767,POINT (-96.0447673 42.3897169)
2,2019,Wright County,1175326,3132,336,293,73,19197,465287,42.733119,-93.735146,POINT (-93.7351465 42.7331189)
3,2019,Winneshiek County,2764203,6775,574,508,101,19191,465284,43.290675,-91.843707,POINT (-91.8437074 43.2906746)
4,2019,Winnebago County,1251960,3657,560,503,56,19189,465283,43.377568,-93.734195,POINT (-93.7341955 43.37756780000001)


## Rural Counties Data

In [13]:
rural_counties_df = pd.read_csv("./datasets/rural_counties.csv")
rural_counties_df.head()

,FIPS code,County Name,Population Estimate 2016,Urban-Rural Designation
0,19001,Adair,"7,676",Rural
1,19003,Adams,"4,028",Rural
2,19005,Allamakee,"14,347",Rural
3,19007,Appanoose,"12,883",Rural
4,19009,Audubon,"6,115",Rural


In [14]:
rural_counties_df = rural_counties_df[["FIPS code", "Urban-Rural Designation"]]
# designation_dict = dict(zip(rural_counties_df["County Name"], rural_counties_df["Urban-Rural Designation"]))

In [15]:
rural_counties_df.head()

,FIPS code,Urban-Rural Designation
0,19001,Rural
1,19003,Rural
2,19005,Rural
3,19007,Rural
4,19009,Rural


In [16]:
totalCounty.shape

(10280, 174)

In [17]:
# temp -> total county
temp = totalCounty.merge(rural_counties_df, how="left", left_on="Fips", right_on="FIPS code", left_index=True)

In [18]:
temp.shape

(10280, 176)

In [19]:
temp.head()

,YearMonth,County,Sale (Dollars),Month,State,ST,Fips,Precincts,Votes,Democrats 08 (Votes),...,Recipients,First Payments,Final Payments,County FIP,GNIS Feature ID,Primary Lat Dec,Primary Long Dec,Primary County Coordinates,FIPS code,Urban-Rural Designation
0,2012-01,Adair County,15988.30,1,Iowa,IA,19001,6.0,3759.0,1924.0,...,314.0,251.0,64.0,19001.0,465190.0,41.330746,-94.470941,POINT (-94.4709413 41.3307464),19001,Rural
0,2012-02,Adair County,17596.63,2,Iowa,IA,19001,6.0,3759.0,1924.0,...,314.0,251.0,64.0,19001.0,465190.0,41.330746,-94.470941,POINT (-94.4709413 41.3307464),19001,Rural
0,2012-03,Adair County,14380.75,3,Iowa,IA,19001,6.0,3759.0,1924.0,...,314.0,251.0,64.0,19001.0,465190.0,41.330746,-94.470941,POINT (-94.4709413 41.3307464),19001,Rural
0,2012-04,Adair County,18161.12,4,Iowa,IA,19001,6.0,3759.0,1924.0,...,314.0,251.0,64.0,19001.0,465190.0,41.330746,-94.470941,POINT (-94.4709413 41.3307464),19001,Rural
0,2012-05,Adair County,18881.41,5,Iowa,IA,19001,6.0,3759.0,1924.0,...,314.0,251.0,64.0,19001.0,465190.0,41.330746,-94.470941,POINT (-94.4709413 41.3307464),19001,Rural


In [20]:
set(list(temp.columns)).difference(set(list(totalCounty.columns)))

{'FIPS code', 'Urban-Rural Designation'}

## Transform Variables

In [21]:
totalCounty["LogSPC"] = np.log(totalCounty["SalesPerCapita"])
totalCounty["LogBachelors"] = np.log(totalCounty["At Least Bachelors's Degree"])
totalCounty["LogPop"] = np.log(totalCounty["Total Population"])
totalCounty["YearsPast2012"] = totalCounty["YearMonth"].str.split("-").str[0].astype(int) - 2012
totalCounty["Adult.smoking"] = totalCounty["Adult.smoking"].replace(np.nan, totalCounty["Adult.smoking"].mean())
totalCounty["BenefitsPerPerson"] = totalCounty["Benefits Paid"] / totalCounty["Total Population"]
totalCounty["Month"] = totalCounty["Month"].astype(str)
totalCounty["Intercept"] = 1
totalCounty["MedianAge*LogPop"] = totalCounty["Median Age"] * totalCounty["LogPop"]
totalCounty["MedianAge*Obesity"] = totalCounty["Median Age"] * totalCounty["Adult.obesity"]
totalCounty["MedianWage*Republicans2012"] = totalCounty["Median Earnings 2010"] * totalCounty["Republicans 2012"]
totalCounty["White*LogPop"] = totalCounty["White"] * totalCounty["LogPop"]

## Linear Regression

In [30]:
print(X.T.shape, y.shape)

(29, 9393) (9393,)


array([ 2.75721258e+01, -8.27838359e-01, -3.01879050e+01, -6.66399272e-01,
        1.01197097e-01, -6.23600447e-02,  2.44257722e-01, -1.04374815e+00,
       -1.11949430e-04,  3.76143759e-02, -6.45884286e-01, -1.51847973e-02,
       -1.34699590e-03, -4.39036165e-02,  6.99702255e-02,  6.35166773e-01,
        8.70769139e-07, -1.33091104e-02,  3.35837714e-01,  1.43882269e-01,
        4.18299973e-01,  4.13403179e-02,  7.31365261e-02,  1.00810881e-01,
        2.29119346e-01,  2.17676508e-01,  1.91245824e-01,  1.76274690e-01,
        1.26737461e-01])

In [29]:
np.linalg.inv(X.T@X)@X.T@y

ValueError: operands could not be broadcast together with shapes (29,9393) (9393,29) 

In [ ]:
np.linalg.inv(X.T@X).shape

In [24]:
cols = ["Intercept", "Month", "Median Age", "Adult.obesity", "Diabetes", "White", 
        "Less Than High School Diploma", "LogBachelors", "LogPop", 
        "Median Earnings 2010", "YearsPast2012", "Adult.smoking", "Republicans 2012", 
        "BenefitsPerPerson", "School Enrollment", "MedianAge*LogPop", 
        "MedianAge*Obesity", "MedianWage*Republicans2012", "White*LogPop"]
X = totalCounty[cols].dropna()
y = np.array(totalCounty['LogSPC'].loc[X.index])
X = pd.get_dummies(X, drop_first=True)
dummy_cols = X.columns
X = np.array(X)

b = np.linalg.inv(X.T@X)@X.T@y

np.sqrt(((np.e**y - np.e**(X@b))**2).mean())

2.495805686940038

In [ ]:
totalCounty[totalCounty["County"] == "Dickinson County"]["SalesPerCapita"].mean()

In [ ]:
totalCounty[totalCounty["County"] != "Dickinson County"]["SalesPerCapita"].mean()

In [ ]:
X.shape

In [ ]:
y.shape

## Cross Validation

In [ ]:
indices = [i for i in range(X.shape[0])]
group_size = X.shape[0] // 4
groups = []

for i in range(3):
    cur_group = random.sample(indices, group_size)
    groups.append(cur_group)
    for ind in cur_group:
        indices.remove(ind)
        
groups.append(indices)

In [ ]:
scores = []
for i in range(4):
    train_inds = sum([groups[j] for j in range(len(groups)) if j != i], [])
    X_train = X[train_inds]
    X_test = X[groups[i]]
    y_train = y[train_inds]
    y_test = y[groups[i]]
    
    b = np.linalg.inv(X_train.T@X_train)@X_train.T@y_train
    scores.append(np.sqrt(((np.e**y_test - np.e**(X_test@b))**2).mean()))
    
np.mean(scores)

## AIC Score

In [ ]:
def get_aic_score(X):
    indices = [i for i in range(X.shape[0])]
    group_size = X.shape[0] // 4
    groups = []

    for i in range(3):
        cur_group = random.sample(indices, group_size)
        groups.append(cur_group)
        for ind in cur_group:
            indices.remove(ind)

    groups.append(indices)

    aic_scores = []
    for i in range(4):
        train_inds = sum([groups[j] for j in range(len(groups)) if j != i], [])
        X_train = X[train_inds]
        X_test = X[groups[i]]
        y_train = y[train_inds]
        y_test = y[groups[i]]

        b = np.linalg.inv(X_train.T@X_train)@X_train.T@y_train
        residuals = np.e**y_test - np.e**(X_test@b)
        aic_scores.append((residuals**2).sum() / np.std(y_test)**2 + 2*X_train.shape[1])

    return np.mean(aic_scores)

get_aic_score(X)

## Calculate R^2

In [ ]:
predictions = np.e**(X@b)
actual = np.e**y

ssTot = ((actual.mean() - actual)**2).sum()
ssRes = ((predictions - actual)**2).sum()

R2 = 1 - ssRes/ssTot
R2

## Slope Confidence Intervals

In [ ]:
bounds = []
t = 1.96
for i, col in enumerate(dummy_cols):
    var = X[:,i]
    se = np.sqrt(((predictions - actual)**2).sum() / (len(actual) - 2)) / np.sqrt(((var - var.mean())**2).sum())
    bounds.append((b[i] - se*t, b[i] + se*t))
    
ci_df = pd.DataFrame({"Variable": dummy_cols, "Bounds": bounds})

ci_df.head()

## Predicted vs. Actual Plot

In [ ]:
res = pd.DataFrame({"Predictions": predictions, "Actual": actual})

In [ ]:
res.plot.scatter("Predictions", "Actual")

## Model Selection

In [ ]:
# remove unimportant variables 
cols = ["Intercept", "Month", "Median Age", "Adult.obesity", "Diabetes", "White", "Less Than High School Diploma", "LogBachelors", "LogPop", "Median Earnings 2010", "YearsPast2012", "Adult.smoking", "Republicans 2012", "School Enrollment", "MedianAge*LogPop", "MedianAge*Obesity", "MedianWage*Republicans2012", "White*LogPop"]
done = False

while not done:
    aic_scores = []
    X = totalCounty[cols].dropna()
    y = np.array(totalCounty['LogSPC'].loc[X.index])
    X = pd.get_dummies(X, drop_first=True)
    X = np.array(X)
    initial_aic = get_aic_score(X)
    print(initial_aic)
    for col in cols: 
        X = totalCounty[cols].dropna().drop(col, axis=1)
        y = np.array(totalCounty['LogSPC'].loc[X.index])
        X = pd.get_dummies(X, drop_first=True)
        X = np.array(X)
        aic_scores.append(get_aic_score(X))
    scores = pd.DataFrame({"Column": cols, "AIC Score": aic_scores})
    if min(aic_scores) > initial_aic:
        done = True
    else:
        display(scores.sort_values("AIC Score"))
        removed_col = scores.sort_values("AIC Score")["Column"].iloc[0]
        print("Removed Column: ", removed_col)
        cols.remove(removed_col)

## Interesting Observations

Dickinson County has much higher sales per capita than other counties, on average. 

In [ ]:
totalCounty[totalCounty["County"] == "Dickinson County"]["SalesPerCapita"].mean()

In [ ]:
totalCounty[totalCounty["County"] != "Dickinson County"]["SalesPerCapita"].mean()

In [ ]:
list(totalCounty.columns)